# Finding a relationship between the category of products and reviews of customers

In [17]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [18]:
df_reviews = pd.read_csv('../data/olist_order_reviews_dataset.csv')
df_orders = pd.read_csv('../data/olist_orders_dataset.csv')
df_items = pd.read_csv('../data/olist_order_items_dataset.csv')
df_products = pd.read_csv('../data/olist_products_dataset.csv')

In [46]:
df_reviews.dtypes

review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object

In [47]:
df_orders.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [49]:
df_items.dtypes

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [52]:
df_products.dtypes

product_id                     object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_photos_qty            float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object

In [91]:
df = df_reviews[['order_id','review_score']].merge(df_orders[['order_id']], on='order_id', how='left').merge(df_items[['order_id','product_id','price']], on='order_id', how='left').merge(df_products[['product_id','product_category_name']], on='product_id', how='left')

# CLEAN DATA
# Drop duplicate orders with same order_id
df = df.drop_duplicates(subset = 'order_id', ignore_index = True) 
# Get rid of products without category
df = df.dropna(axis=0, how='any')
df.head()

,order_id,review_score,product_id,price,product_category_name
0,73fc7af87114b39712e6da79b0a377eb,4,fd25ab760bfbba13c198fa3b4f1a0cd3,185.00,esporte_lazer
1,a548910a1c6147796b98fdf73dbeba33,5,be0dbdc3d67d55727a65d4cd696ca73c,79.79,informatica_acessorios
2,f9e4b658b201a9f2ecdecbb34bed034b,5,d1c427060a0f73f6b889a5c7c61f2ac4,149.00,informatica_acessorios
3,658677c97b385a9be170737859d3511b,5,52c80cedd4e90108bf4fa6a206ef6b03,179.99,ferramentas_jardim
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,3880d25d502b15b1de6fddc42ad1d67a,1199.00,esporte_lazer


# To check which category has the highest and lowest average review score

In [97]:
# Sort data by category
dfcat = df[['product_category_name', 'review_score']]
dfcat = dfcat.set_index('product_category_name')
dfcat.sort_values(by='product_category_name')

,review_score
product_category_name,
agro_industria_e_comercio,5
agro_industria_e_comercio,4
agro_industria_e_comercio,1
agro_industria_e_comercio,5
agro_industria_e_comercio,4
...,...
utilidades_domesticas,5
utilidades_domesticas,4
utilidades_domesticas,4


In [103]:
dfcat.groupby(['product_category_name']).size()

product_category_name
agro_industria_e_comercio     182
alimentos                     444
alimentos_bebidas             224
artes                         198
artes_e_artesanato             22
                             ... 
sinalizacao_e_seguranca       140
tablets_impressao_imagem       77
telefonia                    4182
telefonia_fixa                217
utilidades_domesticas        5829
Length: 73, dtype: int64

Sort data by category, including the number of items on the side

In [106]:
# Calculate the average review score for each category
avgscore = dfcat.groupby(['product_category_name']).mean()

avgscore.head()

,review_score
product_category_name,
agro_industria_e_comercio,4.021978
alimentos,4.247748
alimentos_bebidas,4.366071
artes,4.030303
artes_e_artesanato,4.136364


In [107]:
# Sort according to highest average review score to lowest average review score
avgscore = avgscore.sort_values(by='review_score', ascending=False)
avgscore

,review_score
product_category_name,
cds_dvds_musicais,4.666667
fashion_roupa_infanto_juvenil,4.500000
livros_interesse_geral,4.459725
fashion_esporte,4.400000
livros_importados,4.384615
...,...
fashion_roupa_masculina,3.678571
moveis_escritorio,3.615810
portateis_cozinha_e_preparadores_de_alimentos,3.428571


In [108]:
print("Highest average", avgscore.iloc[0])
print()
print("Lowest average", avgscore.iloc[-1])

Highest average review_score    4.666667
Name: cds_dvds_musicais, dtype: float64

Lowest average review_score    2.5
Name: seguros_e_servicos, dtype: float64


According to Google Translate, these are "CDs, DVDs, Musicals" and "Insurances & Services" respectively.

# See which category has the most quantity sold

In [111]:
quantity = dfcat.groupby(['product_category_name']).size().sort_values(ascending=False)
quantity

product_category_name
cama_mesa_banho                  9311
beleza_saude                     8796
esporte_lazer                    7681
informatica_acessorios           6660
moveis_decoracao                 6355
                                 ... 
la_cuisine                         12
cds_dvds_musicais                  12
fashion_roupa_infanto_juvenil       8
pc_gamer                            7
seguros_e_servicos                  2
Length: 73, dtype: int64

According to Google Translate, the category that has the most items sold is "Bedding, Duvet & Towel Sets", and the one with least items sold is "Insurances & Services", which means that the average review score for that is not accurate as the N value is not comparable to the other category

# See which category has the highest profits

In [113]:
# Sort data by Category
dfprofit = df[['product_category_name', 'price']]
dfprofit = dfprofit.set_index('product_category_name')
dfprofit.sort_values(by='product_category_name')

,price
product_category_name,
agro_industria_e_comercio,412.00
agro_industria_e_comercio,410.00
agro_industria_e_comercio,469.00
agro_industria_e_comercio,425.00
agro_industria_e_comercio,429.98
...,...
utilidades_domesticas,164.90
utilidades_domesticas,32.00
utilidades_domesticas,120.00


In [154]:
dfprofitaverage = dfprofit.groupby(['product_category_name']).mean('price')
dfprofitaverage.sort_values(by='price', ascending=False)

,price
product_category_name,
pcs,1140.078778
portateis_casa_forno_e_cafe,624.676133
eletrodomesticos_2,484.699399
agro_industria_e_comercio,351.844396
instrumentos_musicais,296.541891
...,...
bebidas,57.709116
cds_dvds_musicais,53.333333
fraldas_higiene,46.466296


The category that has the highest profits is PCs, while the category that has the lowest profits is from hotel bookings.